In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import os, time

def scrape(path, headless=False):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--hide-scrollbars")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(path)
    driver.maximize_window()

    # Login Steps
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'username'))).send_keys("fourbrotherstrading@icloud.com")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'password'))).send_keys("Sultanmirza1501#")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/button[text()="Sign in"]'))).click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'start-btn'))).click()
    except:
        print("Already logged in / No popup")

    return driver


url = "https://www.centralcarauctions.com/portal/auction/buyer/webauction/auction/838"
driver = scrape(url, headless=False)

os.makedirs("live_html", exist_ok=True)
os.makedirs("screenshots", exist_ok=True)

previous_title = ""
file_counter = 1

try:
    while True:
        if not driver.window_handles:
            print("Browser closed. Exiting.")
            break

        try:
            # 👉 Corrected XPATH (col-lg-9)
            try:
                car_title_elem = WebDriverWait(driver, 4).until(
                    EC.presence_of_element_located((By.XPATH, './/div[contains(@class,"col-lg-9")]/h1'))
                )
                current_title = car_title_elem.text.strip()
            except TimeoutException:
                print("🔄 Waiting for car data...")
                time.sleep(1)
                continue

            if current_title != previous_title:
                previous_title = current_title
                print(f"\n📌 NEW VEHICLE: {current_title}")

                # Extract Registration
                try:
                    reg_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((
                            By.XPATH,
                            '//table[contains(@class,"vehicle-table")]//th[text()="Registration"]/following-sibling::td'
                        ))
                    )
                    reg_number = reg_element.text.strip().replace(" ", "_")
                except:
                    reg_number = current_title[:10].replace(" ", "_")

                # FILE NAMES
                html_file = os.path.join("live_html", f"{file_counter}_{reg_number}.html")
                screenshot_file = os.path.join("screenshots", f"{file_counter}_{reg_number}.png")

                # SAVE HTML
                with open(html_file, "w", encoding="utf-8") as f:
                    f.write(driver.page_source)
                print(f"💾 HTML SAVED: {html_file}")

                # SAVE SCREENSHOT
                driver.save_screenshot(screenshot_file)
                print(f"📸 SCREENSHOT SAVED: {screenshot_file}")

                file_counter += 1
            
            time.sleep(2)

        except StaleElementReferenceException:
            print("♻️ Stale element — retrying...")
            continue

        except Exception as e:
            print("⚠️ Waiting (temporary issue)...")
            time.sleep(2)
            continue

except KeyboardInterrupt:
    print("⛔ Stopped by user.")
finally:
    driver.quit()
    print("🚪 Browser closed.")



📌 NEW VEHICLE: Lot 62: Ford Fiesta
💾 HTML SAVED: live_html\1_SY18HTX.html
📸 SCREENSHOT SAVED: screenshots\1_SY18HTX.png

📌 NEW VEHICLE: Lot 63: Kia Sportage
💾 HTML SAVED: live_html\2_SD65URB.html
📸 SCREENSHOT SAVED: screenshots\2_SD65URB.png

📌 NEW VEHICLE: Lot 64: Fiat 500l
💾 HTML SAVED: live_html\3_ND69WHK.html
📸 SCREENSHOT SAVED: screenshots\3_ND69WHK.png
♻️ Stale element — retrying...

📌 NEW VEHICLE: Lot 56: Vauxhall Mokka X
💾 HTML SAVED: live_html\4_SE18NTC.html
📸 SCREENSHOT SAVED: screenshots\4_SE18NTC.png

📌 NEW VEHICLE: Lot 60: Nissan Juke
💾 HTML SAVED: live_html\5_KE19FBF.html
📸 SCREENSHOT SAVED: screenshots\5_KE19FBF.png

📌 NEW VEHICLE: Lot 64: Fiat 500l
💾 HTML SAVED: live_html\6_ND69WHK.html
📸 SCREENSHOT SAVED: screenshots\6_ND69WHK.png

📌 NEW VEHICLE: Lot 65: BMW 4 Series
💾 HTML SAVED: live_html\7_YK68FFA.html
📸 SCREENSHOT SAVED: screenshots\7_YK68FFA.png

📌 NEW VEHICLE: Lot 66: Volvo XC60
💾 HTML SAVED: live_html\8_KW17YHY.html
📸 SCREENSHOT SAVED: screenshots\8_KW17YHY.png